In [1]:
!pip install langchain
!pip install langchain-community
!pip install langchain-experimental
!pip install google-generativeai
!pip install langchain-google-genai
!pip install pypdf
!pip install unstructured
!pip install openpyxl
!pip install docx2txt
!pip insttall python-magic
!pip install python-pptx
!pip install faiss-cpu
!pip install tiktoken
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 19.1 MB/s eta 0:0

In [102]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.agents.openai_functions_agent.base import create_openai_functions_agent
from langchain.agents.agent import AgentExecutor
from langchain_community.tools.tavily_search.tool import TavilySearchResults
import os
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.document_loaders import WebBaseLoader
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.tools.retriever import create_retriever_tool

In [83]:
loader = WebBaseLoader("https://python.langchain.com/docs/expression_language/")
document = loader.load()

In [84]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
chunks = text_splitter.split_documents(document)

In [85]:
embeddings=GoogleGenerativeAIEmbeddings(model='models/embedding-001')
vector_db=FAISS.from_documents(chunks,embeddings)
retriever=vector_db.as_retriever(similarity_score_threshold=0.4)

In [ ]:
os.environ['GOOGLE_API_KEY']=''
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash-latest')
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant. When searching for information:
    - Use tavily_search_results_json for general web searches
    - Use lcel_search for LangChain-specific information
    Always specify which tool you're using and the exact query."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
os.environ['TAVILY_API_KEY']=''
search=TavilySearchResults()
retriever_tools=create_retriever_tool(
    retriever=retriever,
    name='lcel_search',
    description='"Use this tool when searching for information about Langchain Expression Language (LCEL)."'
)
tools = [search, retriever_tools]

In [98]:
agent=create_openai_functions_agent(llm=llm,prompt=prompt,tools=tools)
agentExecutor=AgentExecutor(agent=agent,tools=tools)

In [99]:
def process_chat(agentExecutor, user_input, chat_history):
    response = agentExecutor.invoke({
        "input": user_input,
        "chat_history": chat_history
    })
    return response["output"]

if __name__ == '__main__':
    chat_history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break

        response = process_chat(agentExecutor, user_input, chat_history)
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=response))

        print("Assistant:", response)

You: 
Assistant: I need more information to answer your question.  Please ask me something specific.

You: what is lcel
Assistant: I'm using the `lcel_search` tool to look up information about LCEL.  However, I need a query to use with that tool.  I'll need to formulate a query to get the information you are looking for.  What specifically about LCEL are you interested in?  For example, are you interested in its definition, its uses, or examples of its syntax?

You: defenition LCEL
Assistant: LangChain Expression Language (LCEL) is a syntax for orchestrating LangChain components.  It's most useful for simpler applications.  The response also mentions that it's used with various LangChain features like document loaders, few-shot prompting, and key-value stores.

You: how about langchain graph?
Assistant: I cannot provide a definition for "Langchain Graph" using the available tools.  There is no information about a specific Langchain component or feature with that name in the provided to